# ConveyorGuard - 00. EDA (Exploratory Data Analysis)

**AIHub #71802 이송장치 멀티모달 데이터셋 탐색적 분석**

- **Accelerator: GPU T4 x2**
- 입력: AIHub 원본 데이터 (Training/Validation)

---

### 분석 목차
```
1. 데이터셋 구조 파악
2. 센서 분포 분석 (클래스별)
3. 클래스 불균형 분석
4. 센서 간 상관관계
5. 시계열 패턴 분석
6. 열화상 이미지 분석
7. 외부환경 영향 분석
8. 분석 요약 및 모델 설계 시사점
```

## 1. 환경 설정

In [ ]:
import os
import sys
import platform
import json
import time
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import warnings
warnings.filterwarnings('ignore')

# 시각화
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.io as pio
pio.renderers.default = 'notebook'

# 환경 정보
print(f"Python: {sys.version}")
try:
    import torch
    print(f"PyTorch: {torch.__version__}")
    print(f"CUDA: {torch.version.cuda}")
    if torch.cuda.is_available():
        for i in range(torch.cuda.device_count()):
            props = torch.cuda.get_device_properties(i)
            # 버전 호환성 처리 (total_mem → total_memory)
            mem = getattr(props, 'total_memory', None) or getattr(props, 'total_mem', 0)
            print(f"GPU {i}: {props.name} ({mem / 1024**3:.1f} GB)")
except ImportError:
    print("PyTorch: not installed")

print(f"\n✅ 라이브러리 로드 완료")

# 전체 실행 시간 측정 시작
_notebook_start = time.time()

In [ ]:
# 경로 설정
DATA_DIR = Path('/kaggle/input/conveyorguard-aihub')
if not DATA_DIR.exists():
    DATA_DIR = Path('/kaggle/input')  # fallback

print(f'\U0001f4c2 데이터 폴더: {DATA_DIR}')
print(f'  Training: {(DATA_DIR / "Training").exists()}')
print(f'  Validation: {(DATA_DIR / "Validation").exists()}')

# 상수
SENSOR_NAMES = ['NTC', 'PM1.0', 'PM2.5', 'PM10', 'CT1', 'CT2', 'CT3', 'CT4']
STATE_LABELS = {0: '정상', 1: '경미', 2: '중간', 3: '심각'}
STATE_COLORS = {0: '#10B981', 1: '#F59E0B', 2: '#F97316', 3: '#EF4444'}

def detect_source_label(parent_dir: Path):
    """'원천' 포함 → source, '라벨' 포함 → label 로 자동 분류"""
    folders = sorted(parent_dir.iterdir())
    source, label = None, None
    for f in folders:
        name = f.name
        # 다양한 키워드 패턴 지원
        if '원천' in name or 'source' in name.lower() or '01.' in name:
            source = f
        elif '라벨' in name or 'label' in name.lower() or '02.' in name:
            label = f
    
    if source is None or label is None:
        print(f'  ⚠️ 키워드 감지 실패, 정렬 순서 사용: {[f.name for f in folders]}')
        source, label = folders[0], folders[1]
    return source, label

train_dir = DATA_DIR / 'Training'
source_dir, label_dir = detect_source_label(train_dir)
print(f'\n  Train source: {source_dir.name}')
print(f'  Train label:  {label_dir.name}')

val_dir = DATA_DIR / 'Validation'
val_source_dir, val_label_dir = detect_source_label(val_dir)
print(f'  Val source:   {val_source_dir.name}')
print(f'  Val label:    {val_label_dir.name}')

## 2. 데이터셋 구조 파악

In [ ]:
# 세션(ZIP 쌍) 탐색 함수
def find_zip_pairs(source_dir: Path, label_dir: Path):
    """원천/라벨 ZIP 파일(폴더) 쌍 찾기"""
    def get_key(filename):
        stem = filename.stem
        if stem[:3] in ['TS_', 'TL_', 'VS_', 'VL_']:
            return stem[3:]
        return stem
    source_zips = {get_key(z): z for z in source_dir.iterdir() if z.name.endswith('.zip') or z.is_dir()}
    label_zips = {get_key(z): z for z in label_dir.iterdir() if z.name.endswith('.zip') or z.is_dir()}
    pairs = []
    for key in source_zips:
        if key in label_zips:
            pairs.append((source_zips[key], label_zips[key]))
    return pairs

# Training/Validation 쌍 찾기 (cell 3의 detect_source_label 결과 사용)
train_pairs = find_zip_pairs(source_dir, label_dir)
val_pairs = find_zip_pairs(val_source_dir, val_label_dir)

assert len(train_pairs) > 0, f"train_pairs 0개! source={source_dir}, label={label_dir}"
assert len(val_pairs) > 0, f"val_pairs 0개! source={val_source_dir}, label={val_label_dir}"

print(f'📊 데이터셋 규모:')
print(f'   Training 세션: {len(train_pairs)}개')
print(f'   Validation 세션: {len(val_pairs)}개')
print(f'   전체 세션: {len(train_pairs) + len(val_pairs)}개')

In [ ]:
# 디버깅: 첫 번째 세션 쌍의 실제 구조 확인
src, lbl = train_pairs[0]

print(f"=== 첫 번째 세션 쌍 ===")
print(f"source: {src}")
print(f"  is_dir: {src.is_dir()}")
print(f"  is_file: {src.is_file()}")
print(f"  suffix: {src.suffix}")
print(f"label:  {lbl}")
print(f"  is_dir: {lbl.is_dir()}")
print(f"  is_file: {lbl.is_file()}")
print(f"  suffix: {lbl.suffix}")

# 디렉토리인 경우 내부 파일 확인
if src.is_dir():
    files = list(src.iterdir())[:10]
    print(f"\nsource 내부 파일 ({len(list(src.iterdir()))}개 중 10개):")
    for f in files:
        print(f"  {f.name} (dir={f.is_dir()}, size={f.stat().st_size})")
    
    csv_files = sorted(src.glob('*.csv'))
    bin_files = sorted(src.glob('*.bin'))
    print(f"\n  *.csv: {len(csv_files)}개")
    print(f"  *.bin: {len(bin_files)}개")

if lbl.is_dir():
    json_files = sorted(lbl.glob('*.json'))
    print(f"  *.json: {len(json_files)}개")

# ZIP인 경우
if src.is_file() and src.suffix == '.zip':
    import zipfile
    with zipfile.ZipFile(src) as zf:
        names = zf.namelist()[:10]
        print(f"\nZIP 내부 ({len(zf.namelist())}개 중 10개):")
        for n in names:
            print(f"  {n}")

# 검증
assert len(csv_files) > 0, "❌ CSV 0개: source 폴더가 잘못 지정됨"
assert len(bin_files) > 0, "❌ BIN 0개: source 폴더가 잘못 지정됨"
assert len(json_files) > 0, "❌ JSON 0개: label 폴더가 잘못 지정됨"
print("\n✅ source/label 매핑 정상")

In [ ]:
from tqdm.auto import tqdm  # 진행률 표시

# 세션별 프레임 수 및 라벨 수집
def load_session_metadata(source_path, label_path, max_sessions=None):
    """세션별 메타데이터 수집 (센서 raw + 라벨 + 외부환경)"""
    all_sensors = []    # 전체 센서 프레임
    all_externals = []  # 전체 외부환경 프레임
    all_states = []     # 전체 라벨
    all_images = []     # 열화상 (일부만)
    session_info = []   # 세션 메타

    pairs = find_zip_pairs(source_path, label_path)
    if max_sessions:
        pairs = pairs[:max_sessions]

    for src, lbl in tqdm(pairs, desc="세션 처리"):
        csv_files = sorted(src.glob('*.csv'))
        bin_files = sorted(src.glob('*.bin'))
        json_files = sorted(lbl.glob('*.json'))

        n_frames = min(len(csv_files), len(bin_files), len(json_files))
        if n_frames == 0:
            continue

        session_sensors = []
        session_states = []
        session_externals = []

        for i in range(n_frames):
            try:
                # CSV
                df = pd.read_csv(csv_files[i])
                vals = df.iloc[0, :8].values.astype(np.float32)
                session_sensors.append(vals)
                all_sensors.append(vals)

                # JSON
                with open(json_files[i], 'r', encoding='utf-8') as f:
                    data = json.load(f)

                state = 0
                annotations = data.get('annotations', [])
                if annotations and len(annotations) > 0:
                    tagging = annotations[0].get('tagging', [])
                    if tagging and len(tagging) > 0:
                        state = int(tagging[0].get('state', 0) or 0)
                session_states.append(state)
                all_states.append(state)

                ext_list = data.get('external_data', [])
                ext = ext_list[0] if ext_list else {}
                def gv(key):
                    v = ext.get(key, [{}])
                    return float(v[0].get('value', 0)) if isinstance(v, list) and v else 0.0
                session_externals.append([gv('ex_temperature'), gv('ex_humidity'), gv('ex_illuminance')])
                all_externals.append(session_externals[-1])

                # BIN (처음 3개 세션만 이미지 수집)
                if len(session_info) < 3 and i % 10 == 0:
                    img = np.load(bin_files[i])
                    if img.shape == (120, 160):
                        all_images.append({'state': state, 'image': img, 'session': src.name})
            except:
                continue

        if session_sensors:
            states_arr = np.array(session_states)
            dominant = int(np.bincount(states_arr).argmax())
            session_info.append({
                'session': src.name,
                'n_frames': len(session_sensors),
                'dominant_state': dominant,
                'state_counts': dict(Counter(session_states)),
            })

    return {
        'sensors': np.array(all_sensors),
        'externals': np.array(all_externals),
        'states': np.array(all_states),
        'images': all_images,
        'sessions': session_info
    }

print('📂 메타데이터 수집 중 (Training)...')
train_meta = load_session_metadata(source_dir, label_dir)
print(f'   프레임: {len(train_meta["states"]):,}개, 세션: {len(train_meta["sessions"])}개')

print('📂 메타데이터 수집 중 (Validation)...')
val_meta = load_session_metadata(val_source_dir, val_label_dir)
print(f'   프레임: {len(val_meta["states"]):,}개, 세션: {len(val_meta["sessions"])}개')

# 전체 합치기
all_sensors = np.concatenate([train_meta['sensors'], val_meta['sensors']])
all_externals = np.concatenate([train_meta['externals'], val_meta['externals']])
all_states = np.concatenate([train_meta['states'], val_meta['states']])
all_images = train_meta['images'] + val_meta['images']
all_sessions = train_meta['sessions'] + val_meta['sessions']

print(f'\n✅ 전체 프레임: {len(all_states):,}개, 전체 세션: {len(all_sessions)}개')

In [ ]:
import joblib
import base64
import os
from IPython.display import HTML

# 저장
SAVE_PATH = '/kaggle/working/eda_metadata.joblib'

metadata = {
    'all_sensors': all_sensors,
    'all_externals': all_externals,
    'all_states': all_states,
    'all_images': all_images,
    'all_sessions': all_sessions,
    'train_meta': train_meta,
    'val_meta': val_meta,
}

joblib.dump(metadata, SAVE_PATH, compress=3)

size_mb = os.path.getsize(SAVE_PATH) / 1024 / 1024
print(f'✅ 저장 완료: {size_mb:.1f} MB')

# base64 → 다운로드 링크 생성
with open(SAVE_PATH, 'rb') as f:
    b64 = base64.b64encode(f.read()).decode()

# 클릭하면 파일 다운로드되는 링크
HTML(f'''
<a download="eda_metadata.joblib" 
   href="data:application/octet-stream;base64,{b64}" 
   style="font-size:20px; padding:10px; background:#4CAF50; color:white; text-decoration:none; border-radius:5px;">
   📥 eda_metadata.joblib 다운로드
</a>
''')

In [ ]:
# 세션별 프레임 수 분포 (기존)
session_df = pd.DataFrame(all_sessions)
session_df['state_label'] = session_df['dominant_state'].map(STATE_LABELS)

print('📊 세션별 프레임 수 통계:')
print(session_df['n_frames'].describe().round(1))

# ⭐ 추가: 실제 프레임 단위 클래스 분포
print('\n📊 프레임 단위 클래스 분포 (전체):')
unique, counts = np.unique(all_states, return_counts=True)
for u, c in zip(unique, counts):
    print(f'   {STATE_LABELS[u]}: {c:,}개 ({c/len(all_states)*100:.1f}%)')

fig = make_subplots(rows=1, cols=2, subplot_titles=['세션별 dominant_state', '프레임별 실제 state'])

# 왼쪽: 세션 dominant (기존)
for state_val in sorted(session_df['dominant_state'].unique()):
    subset = session_df[session_df['dominant_state'] == state_val]
    fig.add_trace(go.Histogram(
        x=subset['n_frames'], name=STATE_LABELS[state_val],
        marker_color=STATE_COLORS[state_val]
    ), row=1, col=1)

# 오른쪽: 프레임 실제 분포
fig.add_trace(go.Bar(
    x=[STATE_LABELS[u] for u in unique],
    y=counts,
    marker_color=[STATE_COLORS[u] for u in unique],
    showlegend=False
), row=1, col=2)

fig.update_layout(height=400, width=1000, title='세션 vs 프레임 단위 클래스 비교')
fig.show(renderer='iframe')

In [ ]:
# ⭐ 상태 전이 패턴 분석 (심각 구간 특성 파악)
print('📊 상태 전이 패턴 분석')
print('=' * 50)

# 세션별 상태 전이 카운트
transition_stats = []
for sess in all_sessions:
    counts = sess['state_counts']
    has_severe = counts.get(3, 0) > 0
    transition_stats.append({
        'session': sess['session'],
        'has_severe': has_severe,
        'severe_frames': counts.get(3, 0),
        'severe_ratio': counts.get(3, 0) / sess['n_frames'] * 100,
        'n_frames': sess['n_frames'],
        'dominant': sess['dominant_state']
    })

trans_df = pd.DataFrame(transition_stats)

# 심각 구간 포함 세션 통계
severe_sessions = trans_df[trans_df['has_severe']]
print(f'심각(3) 구간 포함 세션: {len(severe_sessions)}개 / {len(trans_df)}개')
print(f'심각 구간 평균 비율: {severe_sessions["severe_ratio"].mean():.1f}%')
print(f'심각 구간 최대 비율: {severe_sessions["severe_ratio"].max():.1f}%')

# 시각화
fig = px.histogram(
    severe_sessions, x='severe_ratio',
    title='심각(3) 구간 포함 세션의 심각 비율 분포',
    labels={'severe_ratio': '세션 내 심각 비율 (%)'},
    nbins=20
)
fig.update_layout(height=350, width=600)
fig.show(renderer='iframe')

## 3. 클래스 불균형 분석

In [ ]:
# 프레임 단위 클래스 분포
unique, counts = np.unique(all_states, return_counts=True)
class_df = pd.DataFrame({
    'state': [STATE_LABELS[u] for u in unique],
    'count': counts,
    'ratio': (counts / counts.sum() * 100).round(1)
})
print('📊 프레임 단위 클래스 분포:')
print(class_df.to_string(index=False))

fig = make_subplots(rows=1, cols=2, specs=[[{'type': 'bar'}, {'type': 'pie'}]],
                    subplot_titles=['프레임 수', '비율'])
colors = [STATE_COLORS[u] for u in unique]

fig.add_trace(go.Bar(
    x=[STATE_LABELS[u] for u in unique], y=counts,
    marker_color=colors,
    text=[f'{c:,} ({r}%)' for c, r in zip(counts, class_df['ratio'])],
    textposition='outside'
), row=1, col=1)

fig.add_trace(go.Pie(
    labels=[STATE_LABELS[u] for u in unique], values=counts,
    marker_colors=colors, hole=0.4,
    textinfo='label+percent'
), row=1, col=2)

fig.update_layout(title='클래스 불균형 분석', height=400, width=900, showlegend=False)
fig.show(renderer='iframe')  # ← 여기 추가

# 불균형 비율
imbalance_ratio = counts.max() / counts.min()
print(f'\n⚠️ 불균형 비율 (최대/최소): {imbalance_ratio:.1f}:1')
print(f'   → 클래스 가중치 또는 오버샘플링 필요')

## 4. 센서 분포 분석 (클래스별)

In [ ]:
# 센서 DataFrame 생성
sensor_df = pd.DataFrame(all_sensors, columns=SENSOR_NAMES)
sensor_df['state'] = all_states
sensor_df['state_label'] = sensor_df['state'].map(STATE_LABELS)

# 기본 통계
print('\U0001f4ca 센서 기본 통계:')
print(sensor_df[SENSOR_NAMES].describe().round(2))

In [ ]:
# 클래스별 센서 분포 (Box Plot)
fig = make_subplots(rows=2, cols=4, subplot_titles=SENSOR_NAMES)
for idx, sensor in enumerate(SENSOR_NAMES):
    row = idx // 4 + 1
    col = idx % 4 + 1
    for state_val in sorted(sensor_df['state'].unique()):
        subset = sensor_df[sensor_df['state'] == state_val]
        fig.add_trace(go.Box(
            y=subset[sensor], name=STATE_LABELS[state_val],
            marker_color=STATE_COLORS[state_val],
            showlegend=(idx == 0)
        ), row=row, col=col)
fig.update_layout(title='클래스별 센서값 분포 (Box Plot)', height=600, width=1200)
fig.show(renderer='iframe')

In [ ]:
# 클래스별 센서 평균 비교 (Grouped Bar)
mean_by_class = sensor_df.groupby('state_label')[SENSOR_NAMES].mean()
print('📊 클래스별 센서 평균:')
print(mean_by_class.round(2))

fig = go.Figure()
for state_val in sorted(sensor_df['state'].unique()):
    label = STATE_LABELS[state_val]
    vals = mean_by_class.loc[label]
    fig.add_trace(go.Bar(
        name=label, x=SENSOR_NAMES, y=vals,
        marker_color=STATE_COLORS[state_val]
    ))
fig.update_layout(
    title='클래스별 센서 평균값 비교', barmode='group',
    height=450, width=900,
    xaxis_title='센서', yaxis_title='평균값'
)
fig.show(renderer='iframe')

## 5. 센서 간 상관관계

In [ ]:
# 상관 행렬
corr_df = sensor_df[SENSOR_NAMES + ['state']].corr()
fig = ff.create_annotated_heatmap(
    z=corr_df.values.round(2),
    x=list(corr_df.columns),
    y=list(corr_df.index),
    colorscale='RdBu_r',
    showscale=True,
    zmin=-1, zmax=1
)
fig.update_layout(title='센서-라벨 상관 행렬', height=550, width=650)
fig.show(renderer='iframe')

# state와 상관도 높은 센서 순서
state_corr = corr_df['state'].drop('state').abs().sort_values(ascending=False)
print('📊 열화 상태(state)와 상관도 (절대값 기준):')
for name, val in state_corr.items():
    bar = '█' * int(val * 30)
    print(f'   {name:>6}: {val:.3f} {bar}')

## 6. 시계열 패턴 분석

In [ ]:
# 심각(3) 구간 포함 세션의 시계열 + 상태 변화 분석
severe_session = None
for sess in all_sessions:
    if sess['state_counts'].get(3, 0) > 0:
        severe_session = sess
        break

if severe_session:
    src_path = source_dir / severe_session['session']
    if not src_path.exists():
        src_path = val_source_dir / severe_session['session']
    
    # 라벨 경로
    lbl_path = label_dir / severe_session['session'].replace('TS_', 'TL_')
    if not lbl_path.exists():
        lbl_path = val_label_dir / severe_session['session'].replace('VS_', 'VL_')
    
    csv_files = sorted(src_path.glob('*.csv'))
    json_files = sorted(lbl_path.glob('*.json'))
    
    sensors, states = [], []
    for i in range(min(len(csv_files), len(json_files))):
        try:
            df = pd.read_csv(csv_files[i])
            sensors.append(df.iloc[0, :8].values)
            with open(json_files[i], 'r', encoding='utf-8') as f:
                data = json.load(f)
            state = data.get('annotations', [{}])[0].get('tagging', [{}])[0].get('state', 0) or 0
            states.append(int(state))
        except:
            continue
    
    sensors = np.array(sensors)
    states = np.array(states)
    
    # 시각화
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        subplot_titles=['NTC 센서 시계열', '열화 상태 변화'],
                        row_heights=[0.7, 0.3])
    
    fig.add_trace(go.Scatter(
        y=sensors[:, 0], mode='lines', name='NTC',
        line=dict(color='#EF4444')
    ), row=1, col=1)
    
    fig.add_trace(go.Scatter(
        y=states, mode='lines+markers', name='상태',
        line=dict(color='#3B82F6'),
        marker=dict(color=[STATE_COLORS[s] for s in states], size=4)
    ), row=2, col=1)
    
    fig.update_layout(
        title=f'심각 구간 포함 세션: {severe_session["session"]}',
        height=500, width=1000
    )
    fig.update_yaxes(title_text='NTC (°C)', row=1, col=1)
    fig.update_yaxes(title_text='상태', tickvals=[0,1,2,3], 
                     ticktext=['정상','경미','중간','심각'], row=2, col=1)
    fig.update_xaxes(title_text='프레임', row=2, col=1)
    fig.show(renderer='iframe')
    
    print(f'📊 세션: {severe_session["session"]}')
    print(f'📊 상태 분포: {severe_session["state_counts"]}')
else:
    print('⚠️ 심각(3) 구간 포함 세션 없음')

## 7. 열화상 이미지 분석

In [ ]:
# 클래스별 열화상 이미지 비교
if all_images:
    # 클래스별 1개씩 선택
    class_images = {}
    for img_info in all_images:
        s = img_info['state']
        if s not in class_images:
            class_images[s] = img_info
        if len(class_images) == 4:
            break

    n_imgs = len(class_images)
    fig = make_subplots(
        rows=1, cols=n_imgs,
        subplot_titles=[f'{STATE_LABELS[s]} (max={class_images[s]["image"].max():.1f}°C)'
                        for s in sorted(class_images.keys())]
    )

    for idx, (state, info) in enumerate(sorted(class_images.items())):
        fig.add_trace(go.Heatmap(
            z=info['image'], colorscale='Inferno',
            showscale=(idx == n_imgs - 1),
            colorbar=dict(title='°C') if idx == n_imgs - 1 else None
        ), row=1, col=idx + 1)

    fig.update_layout(title='클래스별 열화상 이미지 비교', height=350, width=1100)
    fig.show(renderer='iframe')
else:
    print('⚠️ 열화상 이미지 데이터 없음')

In [ ]:
# 클래스별 열화상 온도 통계
if all_images:
    temp_stats = []
    for info in all_images:
        img = info['image']
        temp_stats.append({
            'state': STATE_LABELS[info['state']],
            'max': img.max(),
            'mean': img.mean(),
            'std': img.std(),
            'min': img.min()
        })
    temp_df = pd.DataFrame(temp_stats)

    print('📊 클래스별 열화상 온도 통계:')
    print(temp_df.groupby('state')[['max', 'mean', 'std']].mean().round(2))

    fig = px.box(
        temp_df, x='state', y='max',
        color='state',
        color_discrete_map={STATE_LABELS[k]: v for k, v in STATE_COLORS.items()},
        title='클래스별 열화상 최고 온도 분포'
    )
    fig.update_layout(height=400, width=600)
    fig.show(renderer='iframe')

## 8. 외부환경 영향 분석

In [ ]:
# 외부환경 DataFrame
ext_names = ['Temperature', 'Humidity', 'Illuminance']
ext_df = pd.DataFrame(all_externals, columns=ext_names)
ext_df['state'] = all_states
ext_df['state_label'] = ext_df['state'].map(STATE_LABELS)

print('📊 외부환경 기본 통계:')
print(ext_df[ext_names].describe().round(2))

In [ ]:
# 클래스별 외부환경 비교
fig = make_subplots(rows=1, cols=3, subplot_titles=ext_names)

for idx, ext in enumerate(ext_names):
    for state_val in sorted(ext_df['state'].unique()):
        subset = ext_df[ext_df['state'] == state_val]
        fig.add_trace(go.Box(
            y=subset[ext], name=STATE_LABELS[state_val],
            marker_color=STATE_COLORS[state_val],
            showlegend=(idx == 0)
        ), row=1, col=idx + 1)

fig.update_layout(title='클래스별 외부환경 분포', height=400, width=1000)
fig.show(renderer='iframe')

In [ ]:
# 외부환경 ↔ state 상관관계
ext_corr = ext_df[ext_names + ['state']].corr()['state'].drop('state').abs().sort_values(ascending=False)
print('📊 외부환경 → 열화상태 상관도:')
for name, val in ext_corr.items():
    bar = '█' * int(val * 40)
    print(f'   {name:>12}: {val:.3f} {bar}')

if ext_corr.max() < 0.1:
    print('\n💡 외부환경은 열화 상태와 상관도가 매우 낮음')
    print('   → 모델에서 보조적 역할만 기대 (FiLM 조건화 등)')
else:
    print(f'\n💡 {ext_corr.idxmax()}가 상대적으로 높은 상관 → 피처로 활용 가치 있음')

## 9. 분석 요약 및 모델 설계 시사점

In [ ]:
print('=' * 70)
print('📋 EDA 분석 요약')
print('=' * 70)

print(f'''
1. 데이터 규모
   - 전체 세션: {len(all_sessions)}개
   - 전체 프레임: {len(all_states):,}개
   - 센서: 8채널 (NTC, PM1.0, PM2.5, PM10, CT1-CT4)

2. 클래스 분포 (프레임 단위)
   - 정상(0): {(all_states==0).sum():,}개 ({(all_states==0).mean()*100:.1f}%)
   - 경미(1): {(all_states==1).sum():,}개 ({(all_states==1).mean()*100:.1f}%)
   - 중간(2): {(all_states==2).sum():,}개 ({(all_states==2).mean()*100:.1f}%)
   - 심각(3): {(all_states==3).sum():,}개 ({(all_states==3).mean()*100:.1f}%)

3. ⭐ 핵심 발견: 심각(3) 클래스 특성
   - 심각이 dominant인 세션: 0개 (전체 세션 중)
   - 심각 구간 포함 세션: {len(severe_sessions)}개
   - → 심각은 세션 전체가 아닌 "특정 구간"에서만 발생
   - → 윈도우 단위 분류가 필수

4. 모델 설계 시사점
   - ❌ 세션 단위 분류: 심각 클래스 탐지 불가
   - ✅ 윈도우 단위 분류: 30프레임 윈도우, stride 10
   - ✅ 상태 전이 패턴: 경미→중간→심각 감지 시 사전 경고
   - 주의: 세션 단위 split 필요 (데이터 누수 방지)
''')

_total_time = time.time() - _notebook_start
print(f"⏱️ 총 실행 시간: {_total_time / 60:.1f}분")
print('\\n✅ EDA 완료! → 01_preprocess.ipynb에서 윈도우 기반 전처리 진행')

In [ ]:
!pip install -q kaleido

In [ ]:
import json
import base64
import os
from IPython.display import HTML

# ============================================================
# 1. EDA 요약 JSON 저장
# ============================================================
eda_summary = {
    'data_info': {
        'total_sessions': len(all_sessions),
        'total_frames': len(all_states),
        'sensors': SENSOR_NAMES,
    },
    'class_distribution': {
        'normal': int((all_states == 0).sum()),
        'minor': int((all_states == 1).sum()),
        'moderate': int((all_states == 2).sum()),
        'severe': int((all_states == 3).sum()),
        'imbalance_ratio': round(imbalance_ratio, 2),
    },
    'severe_analysis': {
        'dominant_sessions': 0,
        'containing_sessions': len(severe_sessions),
    },
    'sensor_importance': {name: round(val, 3) for name, val in state_corr.items()},
    'external_correlation': {name: round(val, 3) for name, val in ext_corr.items()},
}

json_path = '/kaggle/working/eda_summary.json'
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(eda_summary, f, ensure_ascii=False, indent=2)

print('✅ eda_summary.json 저장 완료')

# ============================================================
# 2. JSON 다운로드 링크
# ============================================================
with open(json_path, 'rb') as f:
    b64 = base64.b64encode(f.read()).decode()

print('📥 아래 버튼 클릭하여 다운로드:')

HTML(f'''
<a download="eda_summary.json" href="data:application/octet-stream;base64,{b64}"
   style="display:inline-block; margin:10px 0; padding:15px 30px; 
          background:#4CAF50; color:white; text-decoration:none; 
          border-radius:5px; font-size:18px; font-weight:bold;">
   📥 eda_summary.json 다운로드
</a>
''')

In [ ]:
import shutil
import base64
import os
from IPython.display import HTML

# iframe_figures + eda_summary.json 합쳐서 ZIP
shutil.make_archive('/kaggle/working/eda_all', 'zip', '/kaggle/working', 'iframe_figures')

# JSON도 추가
import zipfile
with zipfile.ZipFile('/kaggle/working/eda_all.zip', 'a') as zf:
    zf.write('/kaggle/working/eda_summary.json', 'eda_summary.json')

# 다운로드 링크
with open('/kaggle/working/eda_all.zip', 'rb') as f:
    b64 = base64.b64encode(f.read()).decode()

zip_size = os.path.getsize('/kaggle/working/eda_all.zip') / 1024 / 1024
print(f'📦 ZIP 파일: {zip_size:.1f} MB')

HTML(f'''
<a download="eda_all.zip" href="data:application/octet-stream;base64,{b64}"
   style="display:inline-block; margin:10px 0; padding:15px 30px; 
          background:#4CAF50; color:white; text-decoration:none; 
          border-radius:5px; font-size:18px; font-weight:bold;">
   📥 eda_all.zip 다운로드 (HTML + JSON)
</a>
''')